In [5]:
from bs4 import BeautifulSoup
import requests
import sys
import time
import pandas as pd
import urllib.robotparser as urobot

header = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_5) AppleWebKit/605.1.15 (KHTML, like Gecko) Safari/605.1.15'}

In [6]:
import urllib

#function to parse the robots.txt of a url
def getRobot(url):
    try:
        robotLink = url + "/robots.txt"
        header = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_5) AppleWebKit/605.1.15 (KHTML, like Gecko) Safari/605.1.15'}
        rp = urobot.RobotFileParser()
        rp.set_url(robotLink)
        rp.read()
        
        return rp         
    except Exception:
        return None


In [7]:
#Class definition of the bfs_crawler
class bfs_crawler:
    def __init__(self, url):
        self.url = url                
        self.robotParser = getRobot(url)     #robot parser to check if a link is valid or not
        self.links_list = []                 #list of allowed links
        
#     def __init__(self):
#         self.url = None
#         self.robotParser = None
#         self.links_list = []
    
    def get_links(self):
        actual_link = self.url
        link_count = 0
        
        while(len(self.links_list) < 200):
            try:
                if("http" not in actual_link):
                    actual_link = "http://" + actual_link
                req = requests.get(actual_link, headers=header)
                
                if(req.status_code == 200):    
                    soup = BeautifulSoup(req.text)
                    pageLinks = soup.findAll("a", href=True)
                    for a in pageLinks:
                        link = a["href"]
                        if(self.robotParser.can_fetch("*", link)):
                            self.links_list.append(link)
                    
            except Exception:
                print("Link inválido!")
            finally: 
                if(link_count >= len(self.links_list)):
                    print("END")
                    return
                actual_link = self.links_list[link_count]
                link_count += 1
                time.sleep(1)